In [1]:
%matplotlib inline

import cvxpy as cvx
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("poster")
sns.set_style("ticks")

ImportError: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.14' not found (required by /content/smishra8/SOFTWARE/anaconda2/lib/python2.7/site-packages/_CVXcanon.so)